### Basic expansion for different organisms

Loading the necessary packages and setting the working directory.

In [1]:
cd(joinpath(homedir(), "Macdocs", "Master", "Internship_JP", "metnetexp", "BioXP"))

using Pkg
Pkg.activate(joinpath(pwd(), "Project.toml"))

using BioXP, ProgressMeter, BenchmarkTools, Profile, JSON, SparseArrays

  Activating project at `~/Macdocs/Master/Internship_JP/metnetexp/BioXP`


Input directory structure, including to-be-used files

In [41]:
#Directory/Folder/File name variables
#Names
organismtype = "methanogens"
sidname = "Contains KEGGID P"
tidname = "targets_Freilich09"
with_sidname = replace(sidname, " " => "_")

#Files
rstructsfile = "master_from_redges-og-submission.json"
seedsfile = "encel_papers_2019.json"
targetsfile = "seeds.json"

#Folders
ridsdir = "rids-methanogens"
seedsdir = "seeds"
targetsdir = "targets"


#All directory paths. No need to change these unless the structure of the directories change
inputdir = joinpath(pwd(),"data","input")

seedsdir = joinpath(inputdir, seedsdir)
targetsdir = joinpath(inputdir, targetsdir)
ridsdir = joinpath(inputdir, ridsdir)

outputdir = joinpath(pwd(),"data","output")
writedir = joinpath(outputdir, with_sidname*"_$organismtype")

#Creates writing directory, if it doesn't exist
!ispath(writedir) && mkpath(writedir);

#All file paths
rstructspath = joinpath(inputdir, rstructsfile)
seedspath = joinpath(seedsdir, seedsfile)
targetspath = joinpath(seedsdir, targetsfile)

printDirectoryPaths (generic function with 1 method)

In [ ]:
"""
This function prints the directory and file paths 
that are used in the script, to check if they are correct
Gets all the arguments in the Main module, 
filters for the ones that end with "dir" or "path"
and prints them in a readable format
"""

function getandprintDirPaths()

    vars_dict = Dict()

    for var in names(Main)

        #Checks if the variable is a directory or file path
        if endswith(string(var), "dir") || endswith(string(var), "path")
            
    
            if endswith(string(var), "dir") 
                tbprinted = join(split(eval(var), "/")[end-2:end], "/") 
                dict_key_end = " Directory: "
                dict_key_beg = uppercasefirst(split(string(var), "dir")[1])
            else
                tbprinted = join(split(eval(var), "/")[end:end], "/")
                dict_key_end = " file path: "
                dict_key_beg = uppercasefirst(split(string(var), "path")[1])
            end

            vars_dict[dict_key_beg*dict_key_end] = tbprinted
            
        end
    end

    for key in keys(vars_dict)
        println(key, vars_dict[key], "\n")
    end
end

getandprintDirPaths()


In [42]:
#Create usable data structures from files based on parser functions
sids = readkeyedids(seedspath)[sidname]
tids = readkeyedids(targetspath)[tidname]

#Get all the seed filepaths and filenames
seed_filepaths = filter(x -> !startswith(x, "."), readdir(ridsdir, join=true))
seed_filenames = filter(x -> !startswith(x, "."), readdir(ridsdir));

#Running the actual analysis
Threads.@threads for (i, file_name) in collect(enumerate(seed_filenames))

    # Why is it needed to create a dir for every seed file?
    write_path = joinpath(writedir, seed_filenames[i])
    #!ispath(write_path) && mkpath(write_path) 

    p = Progress(length(file_name), desc=file_name)
    rids = readids(seed_filepaths[i])
    
    x, t, cids, X, Y = expand(rstructs, rids, sids, tids=tids, write_path=write_path)
    next!(p)

end

In [51]:
# for i in 1:Threads.nthreads()
#     println("i: ", i, "\t Thread ID: ", Threads.threadid()) 
# end

i: 1	 Thread ID: 1
